<h2> Exercici de Recuperació 1 </h2>
<br>- Omnirobot de 3 rodes. 


### 1. Connexió al coppelia


In [98]:
import sim
import time
import math
import numpy as np
def connect(port):
# Establece la conexión a COPPELIA
# El port debe coincidir con el puerto de conexión en VREP  -- DALE AL PLAY !!!
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


In [99]:
# Conectarse al servidor de COPPELIA
# *** _Hay que ejecutarlo cada vez que se reinicia la simulación ***
clientID = connect(19999)

conectado a 19999


<h3> 2. Controladors</h3>

In [100]:
# Obtenemos los controladores para cada eje de las ruedas y el ultrasonido
def getControladors():
    ret,WCR=sim.simxGetObjectHandle(clientID,'OmniWCR',sim.simx_opmode_blocking)
    ret,WCL=sim.simxGetObjectHandle(clientID,'OmniWCL',sim.simx_opmode_blocking)
    ret,WC=sim.simxGetObjectHandle(clientID,'OmniWC',sim.simx_opmode_blocking)

    # Obtenim també "omniRob" que serà molt util per veure la posició del robot 
    ret, omniRob = sim.simxGetObjectHandle(clientID, 'omniRob', sim.simx_opmode_blocking)

    return WCR, WCL, WC, omniRob


WCR, WCL, WC, omniRob = getControladors()

print(WCR, WCL, WC, omniRob)

21 26 16 15


<h3> 3. Moviments del robot</h3>
<br>Heu de realitzar les següents funcions per realitzar moviments lineals i figures :
<br> 1- Funció per avançar en un determinat temps
<br> 2- Funció per girar 90graus a la dreta (mireu la propietat d'un objecte)
<br> 3- Funció per girar 90 graus a l'esquerra (mireu la propietat d'un objecte)
<br> 4- Funció per realitzar un quadrat
<br> 5- Funció per realitzar un triangle
<br> 6- Funció per realitzar un cercle




<h3> 4 Incorporació de sensors de distancia </h3>

Heu d'utilitzar 4 sensors i colocar a cada costat 1 sensor. Intenteu realitzar un programa per moure el robot dins d'un escenari amb  obstacles ( poden ser parets, objectes del coppelia )  i intentar que el robot detecti els obstacles i els pugui evitar 


<h3> Podeu veure els videos de l'exercici 10 on s'explica com afegir aquests sensors </h3>








In [79]:
# 1- Funció per avançar en un determinat temps

def goForward(clientID, wheel_L, wheel_R, duration):
    speed = 3 # No la passo per parametre perquè l'enunciat només especifica "en un determinat temps"

    # Moviment als controladors especificats
    sim.simxSetJointTargetVelocity(clientID, wheel_L, speed, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_R, -speed, sim.simx_opmode_streaming)

    # Esperar durant el temps definit a "duration"
    time.sleep(duration)

    # Parar després d'esperar
    sim.simxSetJointTargetVelocity(clientID, wheel_L, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_R, 0, sim.simx_opmode_streaming)


In [29]:
# Exemple d'us amb els controladors WCR, WCL:
duration = 3  # Duració (segons)
# goForward(clientID, WCR, WCL, duration)

In [153]:
radio_rueda=0.055
radio_robot=0.29

def phis(velocidad_x, velocidad_y, velocidad_angular):
    phi_1 = (1/radio_rueda) * (0.5*math.sqrt(3)*velocidad_x - 
                               0.5*velocidad_y - 
                               velocidad_angular*radio_robot)
    phi_2 = (1/radio_rueda) * (velocidad_y - velocidad_angular*radio_robot)
    phi_3 = (-1/radio_rueda) * (0.5*math.sqrt(3)*velocidad_x + 
                               0.5*velocidad_y + 
                               velocidad_angular*radio_robot)
    return phi_1, phi_2, phi_3

def giroDerecha(angulo, velocidad_angular=0.5):
    tiempo = angulo / velocidad_angular
    phi_1, phi_2, phi_3 = phis(0, 0, velocidad_angular)
    #aplico fuerzas a las ruedas
    sim.simxSetJointTargetVelocity(clientID, WCL, -phi_1, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID,  WC, -phi_2, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, WCR, -phi_3, sim.simx_opmode_streaming)
    time.sleep(tiempo)
    sim.simxSetJointTargetVelocity(clientID, WCL, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID,  WC, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, WCR, 0, sim.simx_opmode_streaming)
    print(phi_1, phi_2, phi_3, tiempo)

def giroIzquierda(angulo, velocidad_angular=0.5):
    tiempo = angulo / velocidad_angular
    phi_1, phi_2, phi_3 = phis(0, 0, velocidad_angular)
    #aplico fuerzas a las ruedas
    sim.simxSetJointTargetVelocity(clientID, WCL, phi_1, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID,  WC, phi_2, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, WCR, phi_3, sim.simx_opmode_streaming)
    time.sleep(tiempo)
    sim.simxSetJointTargetVelocity(clientID, WCL, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID,  WC, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, WCR, 0, sim.simx_opmode_streaming)
    print(phi_1, phi_2, phi_3, tiempo)

In [154]:
# 2- Funció per girar 90graus a la dreta (mireu la propietat d'un objecte)

# Amb ajuda del codi que em vas enviar:

def turnRight():
    giroDerecha(math.pi/2, 1)


In [155]:
# Exemple d'ús de la funció

clientID = connect(19999)

turnRight()

conectado a 19999
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 1.5707963267948966


In [150]:
# 3- Funció per girar 90 graus a l'esquerra (mireu la propietat d'un objecte)

def turnLeft():
    giroIzquierda(math.pi/2, 1)


In [151]:
# Exemple d'ús de la funció

clientID = connect(19999)

turnLeft()

conectado a 19999
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 1.5707963267948966


In [110]:
# 4- Funció per realitzar un quadrat

# Aprofitant les funcions anteriors, podem fer un quadrat fàcilment

def square(clientID, wheel_1, wheel_2):
    for _ in range(4):
        goForward(clientID, wheel_1, wheel_2, 3)
        turnRight()
        # turnRight(clientID, omniRob, wheel_1, wheel_2, wheel_3)
    

In [111]:
# Exemple d'ús de la funció
clientID = connect(19999)

square(clientID, WCR, WCL)

conectado a 19999
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 1.5707963267948966
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 1.5707963267948966
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 1.5707963267948966
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 1.5707963267948966


In [112]:
# 5- Funció per realitzar un triangle

# Nomes podrem aprofitar la funció "goForward", ja que les altres giren 90 graus, i ara necesitarem que giri 120 en cada gir (360/3 = 120)
# Per simplificar, fare una funció "turn120degrees".

# 2pi = 360
# pi = 180
# pi/2 = 90

# pi/6 = 30

# 90 + 30 = 120 ==> pi/2 + pi/6 = 120 ==> 3pi/6 + pi/6 = 120 ==> 4pi/6 = 120 ==> 2pi/3 = 120

# 360/120 = 3 ==> 2pi/3

# 2pi/3 = 120

def turn120degrees():
    giroDerecha((2*math.pi)/3, 1)

def triangle(clientID, wheel_1, wheel_2):
    for _ in range(3):
        goForward(clientID, wheel_1, wheel_2, 3)
        turn120degrees()
        # turn120degrees(clientID, omniRob, wheel_1, wheel_2, wheel_3)

In [113]:
clientID = connect(19999)

triangle(clientID, WCR, WCL)

conectado a 19999
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 2.0943951023931953
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 2.0943951023931953
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 2.0943951023931953


In [130]:
# 6- Funció per realitzar un cercle

# Per fer aquesta funció, deixem una roda quieta i movem les altres dos per a que descrigui un cercle.
# He aprofitat l'antic codi de turnLeft i turnRight que feien més o menys el mateix, pero sobre si mateix.

def circle(clientID, omniRob, wheel_1, wheel_2):
    speed = 2  

    # Obtenir l'orientació inicial del robot
    ret, orientacio_inicial = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
    
    if ret != sim.simx_return_ok:
        print("Error al obtenir l'orientació inicial")
        return
    
    # Calcular l'orientació objectiu (360 graus a la dreta)
    
    angulo_objetivo = orientacio_inicial[2] # Volem que doni una volta sencera, per tant, acabi amb la mateixa orientació (gir de 2pi)

    # Establir velocitats de les rodes per girar a la dreta
    sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_2, speed, sim.simx_opmode_streaming)
    # sim.simxSetJointTargetVelocity(clientID, wheel_3, speed, sim.simx_opmode_streaming)
    
    # Bucle fins que el robot gira 90 graus a la dreta
    i=0
    while True:
        ret, orientacion_actual = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
        
        if ret != sim.simx_return_ok:
            print("Error al obtenir l'orientació actual")
            break
        
        angulo_actual = orientacion_actual[2]
        
        if abs(angulo_actual - angulo_objetivo) < 0.2 and i > 50: # Quan estem arribant, disminuim velocitat per prioritzar precisió
            speed = 0.5
            # Establir nova velocitats de les rodes per girar a la dreta
            sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, wheel_2, speed, sim.simx_opmode_streaming)


        # print("i: ", i, ", ABS: ", abs(angulo_actual - angulo_objetivo))
        if abs(angulo_actual - angulo_objetivo) < 0.005 and i > 50: # Diferència entre angle objectiu i actual; 0.005 es la "precisió"
            break

        i+=1
    
    # Parar després de girar
    sim.simxSetJointTargetVelocity(clientID, wheel_1, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_2, 0, sim.simx_opmode_streaming)


In [131]:
clientID = connect(19999)

circle(clientID, omniRob, WCR, WCL)

conectado a 19999


In [142]:
# 4 Incorporació de sensors de distancia
# Heu d'utilitzar 4 sensors i colocar a cada costat 1 sensor. Intenteu realitzar un programa per moure el robot dins d'un escenari 
# amb  obstacles ( poden ser parets, objectes del coppelia )  i intentar que el robot detecti els obstacles i els pugui evitar 

def moveAvoidingObstacles(clientID, wheel_1, wheel_2, wheel_3, Front_PS, Left_PS, Right_PS, Rear_PS):
    speed = 3  
    
    # Lectura en mode "blocking" perque "buffer", per algun motiu no em funciona amb sensor tipus "conic" (si amb tipus "ray")
    ret, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = sim.simxReadProximitySensor(clientID, Front_PS, sim.simx_opmode_blocking)
    
    while True:
        posibleBug = True

        while True:
            # Llegir estat sensor frontal
            ret, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = sim.simxReadProximitySensor(clientID, Front_PS, sim.simx_opmode_blocking)
            # print(ret, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector)

            if detectionState:
                print("Object detected")
                break

            posibleBug = False

            # Movimient rodes
            sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, wheel_2, - speed, sim.simx_opmode_streaming)
            
        # Parar després de detectar obstacle
        sim.simxSetJointTargetVelocity(clientID, wheel_1, 0, sim.simx_opmode_streaming)
        sim.simxSetJointTargetVelocity(clientID, wheel_2, 0, sim.simx_opmode_streaming)

        # Girar després de trobar obstacle cap a la direcció sense obstacles
        ret, leftDetectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = sim.simxReadProximitySensor(clientID, Left_PS, sim.simx_opmode_blocking)
        ret, rightDetectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = sim.simxReadProximitySensor(clientID, Right_PS, sim.simx_opmode_blocking)
        ret, rearDetectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = sim.simxReadProximitySensor(clientID, Rear_PS, sim.simx_opmode_blocking)
    
        if(posibleBug):
            turnRight()
            turnRight()

        if(leftDetectionState == False):
            turnLeft()
        elif(rightDetectionState == False):
            turnRight()
        elif(rearDetectionState == False):
            turnRight()
            turnRight()
        else:
            print("Atrapat!")
            break

In [143]:
clientID = connect(19999)

# Obtenim els sensors
ret, Front_PS = sim.simxGetObjectHandle(clientID, 'Front_proximity_sensor', sim.simx_opmode_blocking)
ret, Left_PS = sim.simxGetObjectHandle(clientID, 'Left_proximity_sensor', sim.simx_opmode_blocking)
ret, Right_PS = sim.simxGetObjectHandle(clientID, 'Right_proximity_sensor', sim.simx_opmode_blocking)
ret, Rear_PS = sim.simxGetObjectHandle(clientID, 'Rear_proximity_sensor', sim.simx_opmode_blocking)

moveAvoidingObstacles(clientID, WCR, WCL, WC, Front_PS, Left_PS, Right_PS, Rear_PS)

conectado a 19999
Object detected
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 1.5707963267948966
Object detected
-5.2727272727272725 -5.2727272727272725 -5.2727272727272725 1.5707963267948966


KeyboardInterrupt: 

In [ ]:
# TESTING

# Funció per avançar fins trobar un obstacle
def goForwardUntilObstacle(clientID, wheel_L, wheel_R, Front_PS):
    speed = 1  
    
    # Lectura en mode "blocking" perque "buffer", per algun motiu no em funciona amb sensor tipus "conic" (si amb tipus "ray")
    ret, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = sim.simxReadProximitySensor(clientID, Front_PS, sim.simx_opmode_blocking)
    
    while True:
        # Llegir estat sensor frontal
        ret, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector = sim.simxReadProximitySensor(clientID, Front_PS, sim.simx_opmode_blocking)
        # print(ret, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector)

        if detectionState:
            print("Object detected")
            break

        # Movimient rodes
        sim.simxSetJointTargetVelocity(clientID, wheel_L, speed, sim.simx_opmode_streaming)
        sim.simxSetJointTargetVelocity(clientID, wheel_R, - speed, sim.simx_opmode_streaming)
        
    # Parar després de detectar obstacle
    sim.simxSetJointTargetVelocity(clientID, wheel_L, 0, sim.simx_opmode_streaming)
    sim.simxSetJointTargetVelocity(clientID, wheel_R, 0, sim.simx_opmode_streaming)

clientID = connect(19999)

ret, Front_PS = sim.simxGetObjectHandle(clientID, 'Front_proximity_sensor', sim.simx_opmode_blocking)
ret, Left_PS = sim.simxGetObjectHandle(clientID, 'Left_proximity_sensor', sim.simx_opmode_blocking)
ret, Right_PS = sim.simxGetObjectHandle(clientID, 'Right_proximity_sensor', sim.simx_opmode_blocking)
ret, Rear_PS = sim.simxGetObjectHandle(clientID, 'Rear_proximity_sensor', sim.simx_opmode_blocking)

# goForwardUntilObstacle(clientID, WCR, WCL, Front_PS)

conectado a 19999


## Codi descartat:

In [ ]:

# #exemple de possibles moviments
# velocidad = 1
# time.sleep(2)
# #Avanzar hacia delante durante 3 segundos:
# sim.simxSetJointTargetVelocity(clientID, WCR,velocidad,sim.simx_opmode_streaming)
# sim.simxSetJointTargetVelocity(clientID, WCL,-velocidad,sim.simx_opmode_streaming)
# time.sleep(3)
# #Retroceder hacia atras durante 3 segundos:
# sim.simxSetJointTargetVelocity(clientID, WCR,-velocidad,sim.simx_opmode_streaming)
# sim.simxSetJointTargetVelocity(clientID, WCL,-velocidad,sim.simx_opmode_streaming)
# time.sleep(3)
# #Girar hacia la derecha durante 3 segundos:
# sim.simxSetJointTargetVelocity(clientID, WCR,-velocidad,sim.simx_opmode_streaming)
# sim.simxSetJointTargetVelocity(clientID, WCL,velocidad,sim.simx_opmode_streaming)
# time.sleep(3)
# #Parar:
# sim.simxSetJointTargetVelocity(clientID, WCR,0,sim.simx_opmode_streaming)
# sim.simxSetJointTargetVelocity(clientID, WCL,0,sim.simx_opmode_streaming)
# time.sleep(3)

In [ ]:
# def turnRight(clientID, omniRob, wheel_1, wheel_2, wheel_3):
#     speed = 0.5  # Velocitat de gir lenta per a que s'identifiqui correctament el gir

#     # Obtenir l'orientació inicial del robot
#     ret, orientacio_inicial = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
    
#     # print("orientacio_inicial: ", orientacio_inicial)

#     if ret != sim.simx_return_ok:
#         print("Error al obtenir l'orientació inicial")
#         return
    
#     # Calcular l'orientació objectiu (90 graus a la dreta)
#     angulo_objetivo = orientacio_inicial[2] - (90 * (math.pi / 180))  # Graus a radiants

#     # print("angulo_objetivo: ", angulo_objetivo)
    
#     # Ajustar l'angle objectiu perquè estigui entre -pi i pi]
#     if angulo_objetivo < -math.pi:
#         angulo_objetivo += 2 * math.pi
    
#     # Establir velocitats de les rodes per girar a la dreta
#     sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_2, speed, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_3, speed, sim.simx_opmode_streaming)
    
#     # Bucle fins que el robot gira 90 graus a la dreta
#     while True:
#         ret, orientacion_actual = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
        
#         if ret != sim.simx_return_ok:
#             print("Error al obtenir l'orientació actual")
#             break
        
#         angulo_actual = orientacion_actual[2]
        
#         # print("[While] angulo_actual: ", angulo_actual)
#         # print("[While] dif: ", abs(angulo_actual - angulo_objetivo))

#         if abs(angulo_actual - angulo_objetivo) < 0.1: # Quan estem arribant, disminuim velocitat per prioritzar precisió
#             speed = 0.01
#             # Establir nova velocitats de les rodes per girar a la dreta
#             sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
#             sim.simxSetJointTargetVelocity(clientID, wheel_2, speed, sim.simx_opmode_streaming)
#             sim.simxSetJointTargetVelocity(clientID, wheel_3, speed, sim.simx_opmode_streaming)


#         if abs(angulo_actual - angulo_objetivo) < 0.005: # Diferència entre angle objectiu i actual; 0.01 es la "precisió"
#             break
    
#     # Parar després de girar
#     sim.simxSetJointTargetVelocity(clientID, wheel_1, 0, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_2, 0, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_3, 0, sim.simx_opmode_streaming)

In [ ]:
# def turnLeft(clientID, omniRob, wheel_1, wheel_2, wheel_3):
#     speed = 0.4  # Velocitat de gir lenta per a que s'identifiqui correctament el gir

#     # Obtenir l'orientació inicial del robot
#     ret, orientacio_inicial = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
    
#     # print("orientacio_inicial: ", orientacio_inicial)

#     if ret != sim.simx_return_ok:
#         print("Error al obtenir l'orientació inicial")
#         return
    
#     # Calcular l'orientació objectiu (90 graus a la ESQUERRA = 360-90 = 270 graus)
#     angulo_objetivo = orientacio_inicial[2] - (270 * (math.pi / 180))  # Graus a radiants

#     # print("angulo_objetivo: ", angulo_objetivo)
    
#     # Ajustar l'angle objectiu perquè estigui entre -pi i pi]
#     if angulo_objetivo < -math.pi:
#         angulo_objetivo += 2 * math.pi
    
#     # Establir velocitats de les rodes per girar a la ESQUERRA
#     sim.simxSetJointTargetVelocity(clientID, wheel_1, -speed, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_2, -speed, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_3, -speed, sim.simx_opmode_streaming)
    
#     # Bucle fins que el robot gira 90 graus a la ESQUERRA
#     while True:
#         ret, orientacion_actual = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
        
#         if ret != sim.simx_return_ok:
#             print("Error al obtenir l'orientació actual")
#             break
        
#         angulo_actual = orientacion_actual[2]
        
#         # print("[While] angulo_actual: ", angulo_actual)
#         # print("[While] dif: ", abs(angulo_actual - angulo_objetivo))

#         if abs(angulo_actual - angulo_objetivo) < 0.1: # Quan estem arribant, disminuim velocitat per prioritzar precisió
#             speed = 0.01
#             # Establir nova velocitats de les rodes per girar a la dreta
#             sim.simxSetJointTargetVelocity(clientID, wheel_1, -speed, sim.simx_opmode_streaming)
#             sim.simxSetJointTargetVelocity(clientID, wheel_2, -speed, sim.simx_opmode_streaming)
#             sim.simxSetJointTargetVelocity(clientID, wheel_3, -speed, sim.simx_opmode_streaming)

#         if abs(angulo_actual - angulo_objetivo) < 0.005: # Diferència entre angle objectiu i actual; 0.01 es la "precisió"
#             break
    
#     # Parar després de girar
#     sim.simxSetJointTargetVelocity(clientID, wheel_1, 0, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_2, 0, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_3, 0, sim.simx_opmode_streaming)

In [ ]:
# def turn120degrees(clientID, omniRob, wheel_1, wheel_2, wheel_3):
#     speed = 0.4  # Velocitat de gir lenta per a que s'identifiqui correctament el gir

#     # Obtenir l'orientació inicial del robot
#     ret, orientacio_inicial = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
    
#     # print("orientacio_inicial: ", orientacio_inicial)

#     if ret != sim.simx_return_ok:
#         print("Error al obtenir l'orientació inicial")
#         return
    
#     # Calcular l'orientació objectiu (120 graus a la DRETA = 360/3 = 120 graus)
#     angulo_objetivo = orientacio_inicial[2] - (120 * (3.14159265 / 180))  # Graus a radiants

#     # print("angulo_objetivo: ", angulo_objetivo)
    
#     # Ajustar l'angle objectiu perquè estigui entre -pi i pi]
#     if angulo_objetivo < -3.14159265:
#         angulo_objetivo += 2 * 3.14159265
    
#     # Establir velocitats de les rodes per girar a la DRETA
#     sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_2, speed, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_3, speed, sim.simx_opmode_streaming)
    
#     # Bucle fins que el robot gira 120 graus a la DRETA
#     while True:
#         ret, orientacion_actual = sim.simxGetObjectOrientation(clientID, omniRob, -1, sim.simx_opmode_blocking)
        
#         if ret != sim.simx_return_ok:
#             print("Error al obtenir l'orientació actual")
#             break
        
#         angulo_actual = orientacion_actual[2]
        
#         # print("[While] angulo_actual: ", angulo_actual)
#         # print("[While] dif: ", abs(angulo_actual - angulo_objetivo))

#         if abs(angulo_actual - angulo_objetivo) < 0.15: # Quan estem arribant, disminuim velocitat per prioritzar precisió
#             speed = 0.05
#             # Establir nova velocitats de les rodes per girar a la dreta
#             sim.simxSetJointTargetVelocity(clientID, wheel_1, speed, sim.simx_opmode_streaming)
#             sim.simxSetJointTargetVelocity(clientID, wheel_2, speed, sim.simx_opmode_streaming)
#             sim.simxSetJointTargetVelocity(clientID, wheel_3, speed, sim.simx_opmode_streaming)

#         if abs(angulo_actual - angulo_objetivo) < 0.005: # Diferència entre angle objectiu i actual; 0.01 es la "precisió"
#             break
    
#     Parar després de girar
#     sim.simxSetJointTargetVelocity(clientID, wheel_1, 0, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_2, 0, sim.simx_opmode_streaming)
#     sim.simxSetJointTargetVelocity(clientID, wheel_3, 0, sim.simx_opmode_streaming)
